**1. Import dataset**

In [22]:
from dataset import UITVSFCDataset

dataset = UITVSFCDataset(root_dir='data/UIT-VSFC').load_data()
print(dataset['train'][100])

{'sentence': 'tôi cảm thấy giảng viên dạy rất tốt .', 'sentiment': '2', 'topic': '0'}


In [23]:
sentiment_labels = {
    0: 'negative',
    1: 'neutral',
    2: 'positive'
}

topic_labels = {
    0: 'lecturer',
    1: 'training program',
    2: 'facility',
    3: 'other'
}

emoji_labels = {
    ':))': 'colonsmilesmile',
    ':)':  'colonsmile',
	':(':  'colonsad',
	'@@':  'colonsurprise',
	'<3':  'colonlove',
	':d':  'colonsmilesmile',
	':3':  'coloncontemn',
	':v':  'colonbigsmile',
	':_':  'coloncc',
	':p':  'colonsmallsmile',
	'>>':  'coloncolon',
	':">': 'colonlovelove',
	'^^': 'colonhihi',
	':': 'doubledot',
	':(': 'colonsadcolon',
	':’(': 'colonsadcolon',
	':@': 'colondoublesurprise',
	'v.v': 'vdotv',
	'...': 'dotdotdot',
	'/': 'fraction',
	'c#': 'cshrap'
}

**2. Preprocess data**

- Notebook này chỉ triển khai trên mức cơ bản, không sử dụng những thuật toán phức tạp.

- Trong notebook này, pipeline preprocess sẽ có cấu trúc như sau:
    + ***B1***: Chuẩn hóa dữ liệu cơ bản (Basic normalization)
    + ***B2***: Tokenize dữ liệu bằng SpaCy (Tokenization)
    + ***B3***: Tăng cường dữ liệu (Data augmentation)
    + ***B4***: Hậu chuẩn hóa dữ liệu (Post-processing / Normalization)

**2.1. Basic Normalization**

- Đầu tiên là tiến hành mapping các emoji trước.

In [24]:
import re

emoji_sorted = sorted(emoji_labels.keys(), key=len, reverse=True)
emoji_regex = "|".join(re.escape(k) for k in emoji_sorted)
pattern = re.compile(emoji_regex, flags=re.IGNORECASE)

In [25]:
def repl(match):
    key = match.group().lower()
    return " " + emoji_labels[key] + " "

def replace_emoji(text):
    return pattern.sub(repl, text)

- Hàm làm sạch và chuẩn hóa dữ liệu

In [26]:
from pydoc import text

vietnamese_pattern = r"[^a-zA-Z0-9\sàáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđĐ]"

def basic_normalize(text:str) -> str:
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = replace_emoji(text)
    # loại bỏ các ký tự đặc biệt ở cuối cùng để tránh mất thông tin từ emoji hehe :)))
    text = re.sub(vietnamese_pattern, "", text, flags=re.UNICODE)
    return text

**2.2. Tokenization**

In [27]:
import spacy

nlp = spacy.blank("vi")

def tokenize_sentence(nlp,text):
    text = basic_normalize(text)
    return [token.text for token in nlp(text)]

In [28]:
example_1 = "gây mê , hay mắng và rất dễ hờn coloncontemn . "
example_2 = "thầy rất dễ thương :)) :3 <3"
print(tokenize_sentence(nlp,example_1))
print(tokenize_sentence(nlp,example_2))

['gây mê', ' ', 'hay', 'mắng', 'và', 'rất', 'dễ', 'hờn', 'coloncontemn']
['thầy', 'rất', 'dễ thương', ' ', 'colonsmilesmile', '  ', 'coloncontemn', '  ', 'colonlove']


**2.2. Data agumentation**

- Các kỹ thuật được dùng trong notebook này là: 